In [0]:
# dbfs:/FileStore/shared_uploads/kumar@vcloudmatesolutions.com/products.csv
# dbfs:/FileStore/shared_uploads/kumar@vcloudmatesolutions.com/sales_orders.csv

In [0]:
sales_ordersDF = spark.read.csv('dbfs:/FileStore/shared_uploads/kumar@vcloudmatesolutions.com/sales_orders.csv',header=True,inferSchema=True)
productsDF     = spark.read.csv('dbfs:/FileStore/shared_uploads/kumar@vcloudmatesolutions.com/products.csv',header=True,inferSchema=True)

In [0]:
display(sales_ordersDF)

order_id,order_date,customer_id,product_id,quantity,total_amount
1,2023-04-15,C045,P040,1,80.19
2,2023-03-02,C004,P045,4,144.64
3,2023-03-11,C024,P034,7,461.16
4,2023-09-29,C048,P039,9,501.03
5,2023-04-25,C008,P043,1,75.96


In [0]:
display(productsDF)

product_id,price
P040,80.19
P045,36.16
P034,65.88
P039,55.67
P043,75.96


In [0]:
display(sales_ordersDF.isStreaming)
display(productsDF.isStreaming)

False

False

In [0]:
sales_ordersDF.write.format('parquet').save('/tmp/salesorders')
productsDF.write.format('parquet').save('/tmp/products')

In [0]:
sales_orders_StreamingDF = spark.readStream.format('parquet') \
                                            .schema(sales_ordersDF.schema) \
                                            .load('/tmp/salesorders')

In [0]:
products_StreamingDF = spark.readStream.format('parquet') \
                                            .schema(productsDF.schema) \
                                            .load('/tmp/products')

In [0]:
display(sales_orders_StreamingDF.isStreaming)
display(products_StreamingDF.isStreaming)

True

True

In [0]:
join_sales_orders_products_StreamingDF = sales_orders_StreamingDF.join(products_StreamingDF,'product_id')

In [0]:
#q = df.writeStream.format("console").start()
#time.sleep(3)
#q.stop()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2299405843140802>, line 1
----> 1 join_sales_orders_products_StreamingDF.show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:1072, in DataFrame.show(self, n, truncate, vertical)
    983 def show(self, n: int = 20, truncate: Union[bool, int] = True, vertical: bool = False) -> None:
    984     """
    985     Prints the first ``n`` rows of the DataFrame to the console.
    986 
   (...)
   1070     name | This is a super l...
   1071     """
-> 10

In [0]:
# query = join_sales_orders_products_StreamingDF.writeStream.format('console').start()

In [0]:
source_checkpointLocation = '/tmp/_checkpoint'
tableName = 'StreamingSalesOrderProductsTable'

In [0]:
query = join_sales_orders_products_StreamingDF.writeStream.format('parquet') \
                                                .option("checkpointLocation",source_checkpointLocation) \
                                                .outputMode('append') \
                                                .table(tableName)
                                                
                                                

In [0]:
totalSalesQuery ='SELECT SUM(total_amount) FROM streamingsalesorderproductstable'

In [0]:
totalSalesQueryResults = spark.sql(totalSalesQuery)

In [0]:
display(totalSalesQueryResults)

sum(total_amount)
1262.98


In [0]:
# query.awaitTermination()